<a href="https://colab.research.google.com/github/lilyli3304/3D-Reconstruction_MachineLearning/blob/main/ResNet_MNIST_Colab_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALL CUDA 8**

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-headless-525-server' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-no-dkms-515-open' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-gds' for glob 'nvidia*'
Note,

In [2]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2023-03-02 07:04:12--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2023-03-02 07:04:12--  https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?Vktxhpix8VatTVzVXyNrhncOpCpH_ylCxoIM7uYPKanuhGX1JULdIi8GS

In [3]:
!find / -name 7fa2af80.pub

/var/cuda-repo-8-0-local-ga2/7fa2af80.pub
find: ‘/proc/59/task/59/net’: Invalid argument
find: ‘/proc/59/net’: Invalid argument


In [4]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

OK


In [5]:
!apt-get update
!apt-get install cuda-8-0

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:3 file:/var/cuda-repo-8-0-local-ga2  Release.gpg [819 B]
Get:3 file:/var/cuda-repo-8-0-local-ga2  Release.gpg [819 B]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 file:/var/cuda-repo-8-0-local-ga2  Packages [22.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:1

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


**IMPORT TORCH**

In [8]:
import torch
import torch.nn as nn
torch.cuda.is_available()

True

**ResNet Models**

In [9]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1,stride=1,padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=stride,padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1,stride=1,padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

# layers - number of times that the ResNet will use the above defined "block"
# ResNet50 - layers = [3,4,6,3]
# corresponding strides are [1,2,2,2]
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet layers
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes)

    def forward(self, x):
        # In the paper, layer name called - conv1, conv2_x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # In the paper, layer name called - conv2_x
        x = self.layer1(x)
        # In the paper, layer name called - conv3_x
        x = self.layer2(x)
        # In the paper, layer name called - conv4_x
        x = self.layer3(x)
        # In the paper, layer name called - conv5_x
        x = self.layer4(x)

        x = self.avgpool(x)
        # before sending into the fully-connected layer, we need to flatten the data
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        # below is for the size of the identity to be added into the current layer
        # is compatible with the size of the expected output of the current layer
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1,
                                                          stride=stride),
                                                nn.BatchNorm2d(out_channels*4))

        # after this layer, number of channels will change
        # here, self.in_channels = 64, out_channels = 64
        # after the above defined "block", we have the output of out_channels*self.expansion = 64*4 = 256
        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4  #256

        # the above is the end of implementing [1x1 64; 3x3 64; 1x1 256] x 1
        # BELOW implements [1x1 64; 3x3 64; 1x1 256] x 2
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))
        #self.in_channels = 256, out_channels = 64, output channels = out_channels*4 = 256
        #inside the block: 256 -> conv1 -> 64 -> conv2 -> 64 - > conv3 -> 256

        return nn.Sequential(*layers)


def ResNet50(img_channels=1, num_classes=10):
    return ResNet(block, [3,4,6,3], img_channels, num_classes)

def ResNet101(img_channels=3, num_classes=1000):
    return ResNet(block, [3,4,23,3], img_channels, num_classes)

def ResNet152(img_channels=3, num_classes=1000):
    return ResNet(block, [3,8,36,3], img_channels, num_classes)


**TRAIN & TEST ON MNIST**

In [12]:
#############################################################
# Load Data
import numpy as np
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
from torch.utils.data import DataLoader, Subset # Gives easier data management and creates mini-batches.
import torchvision.datasets as datasets # Has standard datasets that we can import in a nice way.
import torchvision.transforms as transforms # Transformations we can perform on our dataset.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
num_epochs = 10
batch_size = 64
#num_train_samples = 64*256
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
#train_dataset = Subset(train_dataset, np.arange(num_train_samples))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
#test_dataset = Subset(test_dataset, np.arange(num_train_samples))
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


#############################################################
# Initialize Network
model = ResNet50().to(device)

criterion = nn.CrossEntropyLoss()       # computing loss - using cross entropy loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


#############################################################
# Train Network
# go through each batch in the train loader, targets are the labelled data for each image
# batch_idx in this example has the range of [0,937]
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # debug
        print(batch_idx, loss)

        # backward
        optimizer.zero_grad()   # no backward propagation in this example, hence we set zero gradient of backward propagation for each batch
        loss.backward()

        # gradient descent or adam step
        optimizer.step()        # update the weights based on the gradients computed from the loss.backward() function


#############################################################
# Check Accuracy on Training & Test to see How Good Our Model is
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)      # predictions are predict digit labels
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


#def test():
#    net = ResNet152()
#    # 2 images, 3 img_channels - RGB, 224x224 pixels
#    x = torch.randn(2, 3, 224, 224)
#    y = net(x).to('cuda')
#    print(y.shape)
#test()

流式输出内容被截断，只能显示最后 5000 行内容。
632 tensor(0.2366, device='cuda:0', grad_fn=<NllLossBackward0>)
633 tensor(0.3703, device='cuda:0', grad_fn=<NllLossBackward0>)
634 tensor(0.0793, device='cuda:0', grad_fn=<NllLossBackward0>)
635 tensor(0.1017, device='cuda:0', grad_fn=<NllLossBackward0>)
636 tensor(0.4700, device='cuda:0', grad_fn=<NllLossBackward0>)
637 tensor(0.2481, device='cuda:0', grad_fn=<NllLossBackward0>)
638 tensor(0.1986, device='cuda:0', grad_fn=<NllLossBackward0>)
639 tensor(0.3059, device='cuda:0', grad_fn=<NllLossBackward0>)
640 tensor(0.3635, device='cuda:0', grad_fn=<NllLossBackward0>)
641 tensor(0.1476, device='cuda:0', grad_fn=<NllLossBackward0>)
642 tensor(0.8267, device='cuda:0', grad_fn=<NllLossBackward0>)
643 tensor(0.3035, device='cuda:0', grad_fn=<NllLossBackward0>)
644 tensor(0.1286, device='cuda:0', grad_fn=<NllLossBackward0>)
645 tensor(0.0807, device='cuda:0', grad_fn=<NllLossBackward0>)
646 tensor(0.2781, device='cuda:0', grad_fn=<NllLossBackward0>)
647 tensor(0.